In [ ]:
import logging
import requests
from typing import Optional, Dict, Any, List

logger = logging.getLogger(__name__)

class ZipCodeData(object):
    """A simple data class to hold the results of our geolocation lookup."""
    def __init__(self, state: str, state_abbr: str, city: str, county: str):
        self.state = state
        self.state_abbr = state_abbr
        self.city = city
        self.county = county

    def to_dict(self) -> Dict[str, Any]:
        return {
            "state": self.state,
            "state_abbreviation": self.state_abbr,
            "city": self.city,
            "county": self.county
        }

def get_lat_lon_from_zip(zip_code: str) -> Optional[Dict[str, float]]:
    """
    Step 1: Get latitude and longitude from a ZIP code using a simple API.
    We'll use zippopotam.us for this first step.
    """
    url = f"https://api.zippopotam.us/us/{zip_code}"
    logger.info(f"Fetching lat/lon for ZIP code: {zip_code} from {url}")
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if not data.get("places"):
            logger.warning(f"No places found for ZIP code {zip_code}")
            return None
            
        place = data["places"][0]
        return {
            "latitude": float(place["latitude"]),
            "longitude": float(place["longitude"]),
            "state": place["state"],
            "state_abbr": place["state abbreviation"],
            "city": place["place name"]
        }
    except (requests.RequestException, KeyError, ValueError) as e:
        logger.error(f"Failed to get lat/lon for ZIP {zip_code}: {e}")
        return None

def get_county_from_lat_lon(lat: float, lon: float) -> Optional[str]:
    """
    Step 2: Get county information from latitude and longitude using the
    U.S. Census Bureau's Geocoding API.
    """
    url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
    params = {
        'x': lon,
        'y': lat,
        'benchmark': 'Public_AR_Current',
        'vintage': 'Current_Current',
        'format': 'json'
    }
    logger.info(f"Fetching county for coordinates: (lat={lat}, lon={lon}) from Census Bureau API")
    try:
        response = requests.get(url, params=params, timeout=15)
        response.raise_for_status()
        data = response.json()
        
        geographies = data.get("result", {}).get("geographies", {})
        counties = geographies.get("Counties", [])
        
        if counties:
            county_name = counties[0].get("NAME")
            logger.info(f"Found county: {county_name}")
            return county_name
        else:
            logger.warning(f"No county found for coordinates (lat={lat}, lon={lon})")
            return None
    except (requests.RequestException, KeyError, ValueError) as e:
        logger.error(f"Failed to get county from coordinates: {e}")
        return None

def get_geo_data_from_zip(zip_code: str) -> Optional[ZipCodeData]:
    """
    Orchestrates the two-step process to get state, city, and county from a ZIP code.
    """
    # Step 1: Get Lat/Lon and basic info
    geo_basics = get_lat_lon_from_zip(zip_code)
    if not geo_basics:
        return None
        
    # Step 2: Get County from Lat/Lon
    county = get_county_from_lat_lon(geo_basics["latitude"], geo_basics["longitude"])
    if not county:
        # Fallback: sometimes county info is not available, but we can proceed without it
        logger.warning(f"Could not determine county for ZIP {zip_code}, proceeding without it.")
        county = "Unknown"

    return ZipCodeData(
        state=geo_basics["state"],
        state_abbr=geo_basics["state_abbr"],
        city=geo_basics["city"],
        county=county[:-7]
    )

data = get_geo_data_from_zip("23294")
print(data.county, data.city, data.state, data.state_abbr, sep=",")

In [2]:
import json
import logging
from typing import Optional, Dict, Any, List
from langchain.docstore.document import Document

from insucompass.core.agents.profile_agent import profile_builder
from insucompass.core.agents.query_trasformer import QueryTransformationAgent
from insucompass.core.agents.router_agent import router
from insucompass.services.ingestion_service import IngestionService
from insucompass.core.agents.search_agent import searcher
from insucompass.core.agents.advisor_agent import advisor


from insucompass.services import llm_provider
from insucompass.prompts.prompt_loader import load_prompt
from insucompass.services.vector_store import vector_store_service

llm = llm_provider.get_gemini_llm()
retriever = vector_store_service.get_retriever()
transformer = QueryTransformationAgent(llm, retriever)
ingestor = IngestionService()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 2. Set up the initial state
# This is the profile *after* the user has entered their basic info in the UI.
user_profile = {
    "zip_code": "30303",
    "county": "Fulton",
    "state": "Georgia",
    "age": 34,
    "gender": "Female", # Change to "Male" to test the other logic path
    "household_size": 1,
    "income": 65000,
    "employment_status": "employed_without_coverage",
    "citizenship": "US Citizen",
    "medical_history": None,
    "medications": None,
    "special_cases": None
}

# 3. Start the conversation loop
last_question = ""
turn_count = 0
while turn_count < 10: # Safety break
    turn_count += 1
    print("\n" + "="*20 + f" Turn {turn_count} " + "="*20)
    print("Current Profile State:")
    print(json.dumps(user_profile, indent=2))

    # Get the next question based on the current profile state
    next_question = profile_builder.get_next_question(user_profile)
    print(f"\nInsuCompass Agent: {next_question}")

    if next_question == "PROFILE_COMPLETE":
        print("\n--- Profile building complete! ---")
        break
    
    # Store the question we just asked so we can provide it as context for the update
    last_question = next_question
    
    # Get live input from the person testing the script
    user_answer = input("Your Answer > ")
    if user_answer.lower() == 'quit':
        break
        
    # Use the updater method to get the new profile state
    user_profile = profile_builder.update_profile_with_answer(
        current_profile=user_profile,
        last_question=last_question,
        user_answer=user_answer
    )

print("\n" + "="*20 + " FINAL PROFILE " + "="*20)
print(json.dumps(user_profile, indent=2))

# Test Case 1: Relevant documents
query = input("How can I help you?")

query_with_profile = f"{user_profile} is my complete profile, answer the question: {query}"

retrieved_docs = transformer.transform_and_retrieve(query)

print(retrieved_docs)

print(f"\nTesting with relevant docs for question: '{query_with_profile}'")
is_relevant = router.grade_documents(query_with_profile, retrieved_docs)
print(f"  - Are docs relevant? -> {is_relevant}")

if is_relevant:
    final_answer = advisor.generate_response(query, user_profile, retrieved_docs)
    print(final_answer)
else:
    result_docs = searcher.search(query)
    if result_docs:
        ingestor.ingest_documents(result_docs)
        final_answer = advisor.generate_response(query, user_profile, result_docs)
        print(final_answer)
    else:
        print("Error...Exit..")

2025-07-09 18:15:45,995 - INFO - Initialized LLM Provider: gemini-2.5-flash
2025-07-09 18:15:46,000 - INFO - QueryIntentClassifierAgent initialized successfully.
2025-07-09 18:15:46,001 - INFO - QueryTransformationAgent initialized successfully.



==================== Turn 1 ====================
Current Profile State:
{
  "zip_code": "30303",
  "county": "Fulton",
  "state": "Georgia",
  "age": 34,
  "gender": "Female",
  "household_size": 1,
  "income": 65000,
  "employment_status": "employed_without_coverage",
  "citizenship": "US Citizen",
  "medical_history": null,
  "medications": null,
  "special_cases": null
}


2025-07-09 18:15:47,304 - INFO - LLM returned next step: 'Alright, let's continue building your health profile. To help us understand your needs better, could you please share a bit about your medical history? For example, have you been diagnosed with any chronic conditions like diabetes, high blood pressure, or asthma, or have you had any major surgeries in the past? No need to go into excessive detail, just the key points that might be relevant for your health coverage.'



InsuCompass Agent: Alright, let's continue building your health profile. To help us understand your needs better, could you please share a bit about your medical history? For example, have you been diagnosed with any chronic conditions like diabetes, high blood pressure, or asthma, or have you had any major surgeries in the past? No need to go into excessive detail, just the key points that might be relevant for your health coverage.


2025-07-09 18:15:51,608 - INFO - Successfully updated profile with user's answer.


...

[Document(metadata={'source_id': 91, 'source_url': 'https://www.healthcare.gov/privacy/', 'source_name': 'privacy', 'source_local_path': 'data/raw/source_91_privacy.html', 'merged_chunks_count': 1, 'original_chunk_numbers': [52]}, page_content=". You're about to connect to a third-party site. Select CONTINUE to proceed or CANCEL to stay on this site. Learn more about links to third-party sites . Continue Cancel YouTube This link goes to an external site You are leaving HealthCare.gov. You're about to connect to a third-party site. Select CONTINUE to proceed or CANCEL to stay on this site. Learn more about links to third-party sites . Continue Cancel LinkedIn This link goes to an external site You are leaving HealthCare.gov. You're about to connect to a third-party site. Select CONTINUE to proceed or CANCEL to stay on this site. Learn more about links to third-party sites . Continue Cancel Instagram This link goes to an external site You are leaving HealthCare.gov. You're about to conne

2025-07-09 18:16:34,920 - WARNING - GRADE: Documents are NOT RELEVANT.


  - Are docs relevant? -> False


2025-07-09 18:16:39,079 - INFO - Formulated search query: 'Health insurance options after quitting job COBRA ACA special enrollment period'
2025-07-09 18:16:39,081 - INFO - Performing web search with Tavily for query: 'Health insurance options after quitting job COBRA ACA special enrollment period'
2025-07-09 18:16:42,650 - INFO - Saved web content from https://www.healthinsurance.org/special-enrollment-guide/involuntary-loss-of-coverage-is-a-qualifying-event/ to data/dynamic/https___www.healthinsurance.org_special-enrollment-guide_involuntary-loss-of-coverage-is-a-qualifying-event_.html
2025-07-09 18:16:42,653 - INFO - Saved web content from https://www.healthcare.gov/have-job-based-coverage/if-you-lose-job-based-coverage/ to data/dynamic/https___www.healthcare.gov_have-job-based-coverage_if-you-lose-job-based-coverage_.html
2025-07-09 18:16:42,655 - INFO - Saved web content from https://www.healthcare.gov/unemployed/cobra-coverage/ to data/dynamic/https___www.healthcare.gov_unemploye

Of course. Before you go, I just want to make sure—are you certain you don't have any other questions for me today? I'm here to help if anything else comes to mind.


In [ ]:
import logging
import json
import sqlite3
from typing import List, Dict, Any
from typing_extensions import TypedDict

from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver

# Import all our custom agent and service classes
from insucompass.core.agents.profile_agent import profile_builder
from insucompass.core.agents.query_trasformer import QueryTransformationAgent
from insucompass.core.agents.router_agent import router
from insucompass.services.ingestion_service import IngestionService
from insucompass.core.agents.search_agent import searcher
from insucompass.core.agents.advisor_agent import advisor

from insucompass.services import llm_provider
from insucompass.prompts.prompt_loader import load_prompt
from insucompass.services.vector_store import vector_store_service

llm = llm_provider.get_gemini_llm()
retriever = vector_store_service.get_retriever()
transformer = QueryTransformationAgent(llm, retriever)
ingestor = IngestionService()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- LangGraph State Definition ---
class AgentState(TypedDict):
    """
    Represents the state of our Q&A graph. This state is passed between nodes.
    """
    user_profile: Dict[str, Any]
    question: str
    contextual_question: str
    documents: List[Document]
    conversation_history: List[str]
    generation: str
    is_relevant: bool

def reformulate_query_node(state: AgentState) -> Dict[str, Any]:
    """
    Node 0 (New): Reformulate the user's question to be self-contained.
    """
    logger.info("---NODE: REFORMULATE QUERY---")
    question = state["question"]
    history = state["conversation_history"]
    user_profile = state["user_profile"]

    prompt = load_prompt("query_reformulator")
    history_str = "\n".join(history)
    
    # if not history:
    #     # If there's no history, the question is already standalone
    #     return {"contextual_question": question}
    
    full_prompt = (
        f"{prompt}\n\n"
        f"### User Profile:\n{str(user_profile)}\n\n"
        f"### Conversation History:\n{history_str}\n\n"
        f"### Follow-up Question:\n{question}"
    )
    
    response = llm.invoke(full_prompt)
    contextual_question = response.content.strip()
    logger.info(f"Reformulated question: '{contextual_question}'")
    return {"contextual_question": contextual_question}

# --- Agent Nodes for the Graph ---

def transform_query_node(state: AgentState) -> Dict[str, Any]:
    """Node 1: Transform the user's query and retrieve initial documents."""
    logger.info("---NODE: TRANSFORM QUERY & RETRIEVE---")
    question = state["contextual_question"]
    documents = transformer.transform_and_retrieve(question)
    return {"documents": documents}

def route_documents_node(state: AgentState) -> Dict[str, Any]:
    """Node 2: Grade the retrieved documents to decide if a web search is needed."""
    logger.info("---NODE: ROUTE DOCUMENTS---")
    question = state["question"]
    documents = state["documents"]
    is_relevant = router.grade_documents(question, documents)
    return {"is_relevant": is_relevant}

def search_and_ingest_node(state: AgentState) -> Dict[str, Any]:
    """Node 3 (Fallback Path): Search the web and ingest new information."""
    logger.info("---NODE: SEARCH & INGEST---")
    question = state["question"]
    web_documents = searcher.search(question)
    if web_documents:
        ingestor.ingest_documents(web_documents)
    return {}

def generate_answer_node(state: AgentState) -> Dict[str, Any]:
    """Node 4: Generate the final, conversational answer."""
    logger.info("---NODE: GENERATE ADVISOR RESPONSE---")
    question = state["question"]
    user_profile = state["user_profile"]
    documents = state["documents"]
    generation = advisor.generate_response(question, user_profile, documents)
    
    history = state.get("conversation_history", [])
    history.append(f"User: {question}")
    history.append(f"Agent: {generation}")
    
    return {"generation": generation, "conversation_history": history}

# --- Conditional Edge Logic ---
def should_search_web(state: AgentState) -> str:
    """The conditional edge that directs the graph's flow."""
    logger.info("---ROUTING: Evaluating document relevance---")
    if state["is_relevant"]:
        logger.info(">>> Route: Documents are relevant. Proceeding to generate answer.")
        return "generate"
    else:
        logger.info(">>> Route: Documents are NOT relevant. Proceeding to web search.")
        return "search"

# --- Build and Compile the Graph ---
db_connection = sqlite3.connect("data/checkpoints.db", check_same_thread=False)
memory = SqliteSaver(db_connection)

builder = StateGraph(AgentState)

builder.add_node("reformulate_query", reformulate_query_node)
builder.add_node("transform_query", transform_query_node)
builder.add_node("route_documents", route_documents_node)
builder.add_node("search_and_ingest", search_and_ingest_node)
builder.add_node("generate_answer", generate_answer_node)

builder.set_entry_point("reformulate_query")
builder.add_edge("reformulate_query", "transform_query")
builder.add_edge("transform_query", "route_documents")
builder.add_conditional_edges(
    "route_documents",
    should_search_web,
    {"search": "search_and_ingest", "generate": "generate_answer"},
)
builder.add_edge("search_and_ingest", "generate_answer")
builder.add_edge("generate_answer", END)

app = builder.compile(checkpointer=memory)

# --- Interactive Test Harness (Updated to manage state correctly) ---
if __name__ == '__main__':
    print("--- InsuCompass AI Orchestrator ---")
    
    # --- Phase 1: Profile Building (This part remains the same) ---
    print("Phase 1: Building your health profile...")
    user_profile = {
    "zip_code": "30303",
    "county": "Fulton",
    "state": "Georgia",
    "age": 34,
    "gender": "Female", # Change to "Male" to test the other logic path
    "household_size": 1,
    "income": 65000,
    "employment_status": "employed_without_coverage",
    "citizenship": "US Citizen",
    "medical_history": None,
    "medications": None,
    "special_cases": None
    }
    profile_conversation_history = []
    for turn in range(10):
        question = profile_builder.get_next_question(user_profile)
        if question == "PROFILE_COMPLETE": break
        print(f"\nInsuCompass Agent: {question}")
        profile_conversation_history.append(f"Agent: {question}")
        user_answer = input("Your Answer > ")
        if user_answer.lower() == 'quit': exit()
        profile_conversation_history.append(f"User: {user_answer}")
        user_profile = profile_builder.update_profile_with_answer(
            user_profile, question, user_answer
        )
    print("\n--- Profile building complete! ---")
    print("\nFinal Profile:", json.dumps(user_profile, indent=2))
    
    # --- Phase 2: Q&A Session ---
    print("\n" + "#"*20 + " Q&A Session " + "#"*20)
    print("Your profile is complete. You can now ask me any questions.")
    
    thread_config = {"configurable": {"thread_id": "user-123-session-1"}}
    qna_history = []

    while True:
        user_question = input("\nYour Question > ")
        if user_question.lower() == 'quit': break
        
        inputs = {
            "question": user_question,
            "user_profile": user_profile,
            "conversation_history": qna_history # Pass the current history
        }
        
        print("\n--- InsuCompass is thinking... ---")
        final_state = {}
        for s in app.stream(inputs, config=thread_config):
            node_name = list(s.keys())[0]
            print(f"--- Executed Node: {node_name} ---")
            final_state.update(s)

        final_answer = final_state.get("generate_answer", {}).get("generation", "Sorry, I couldn't generate a response.")
        
        print("\n" + "="*20 + " FINAL ANSWER " + "="*20)
        print(final_answer)
        
        qna_history = final_state.get("generate_answer", {}).get("conversation_history", qna_history)

In [ ]:
import logging
import json
import sqlite3
from typing import List, Dict, Any
from typing_extensions import TypedDict

from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver

# Import all our custom agent and service classes
from insucompass.core.agents.profile_agent import profile_builder
from insucompass.core.agents.query_trasformer import QueryTransformationAgent
from insucompass.core.agents.router_agent import router
from insucompass.services.ingestion_service import IngestionService
from insucompass.core.agents.search_agent import searcher
from insucompass.core.agents.advisor_agent import advisor

from insucompass.services import llm_provider
from insucompass.prompts.prompt_loader import load_prompt
from insucompass.services.vector_store import vector_store_service

llm = llm_provider.get_gemini_llm()
retriever = vector_store_service.get_retriever()
transformer = QueryTransformationAgent(llm, retriever)
ingestor = IngestionService()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- LangGraph State Definition ---
class AgentState(TypedDict):
    """
    Represents the state of our Q&A graph. This state is passed between nodes.
    """
    user_profile: Dict[str, Any]
    question: str
    contextual_question: str
    documents: List[Document]
    conversation_history: List[str]
    generation: str
    is_relevant: bool

def reformulate_query_node(state: AgentState) -> Dict[str, Any]:
    """
    Node 0 (New): Reformulate the user's question to be self-contained.
    """
    logger.info("---NODE: REFORMULATE QUERY---")
    question = state["question"]
    history = state["conversation_history"]
    user_profile = state["user_profile"]

    prompt = load_prompt("query_reformulator")
    history_str = "\n".join(history)
    
    # if not history:
    #     # If there's no history, the question is already standalone
    #     return {"contextual_question": question}
    
    full_prompt = (
        f"{prompt}\n\n"
        f"### User Profile:\n{str(user_profile)}\n\n"
        f"### Conversation History:\n{history_str}\n\n"
        f"### Follow-up Question:\n{question}"
    )
    
    response = llm.invoke(full_prompt)
    contextual_question = response.content.strip()
    logger.info(f"Reformulated question: '{contextual_question}'")
    return {"contextual_question": contextual_question}

# --- Agent Nodes for the Graph ---

def transform_query_node(state: AgentState) -> Dict[str, Any]:
    """Node 1: Transform the user's query and retrieve initial documents."""
    logger.info("---NODE: TRANSFORM QUERY & RETRIEVE---")
    question = state["contextual_question"]
    documents = transformer.transform_and_retrieve(question)
    return {"documents": documents}

def route_documents_node(state: AgentState) -> Dict[str, Any]:
    """Node 2: Grade the retrieved documents to decide if a web search is needed."""
    logger.info("---NODE: ROUTE DOCUMENTS---")
    question = state["question"]
    documents = state["documents"]
    is_relevant = router.grade_documents(question, documents)
    return {"is_relevant": is_relevant}

def search_and_ingest_node(state: AgentState) -> Dict[str, Any]:
    """Node 3 (Fallback Path): Search the web and ingest new information."""
    logger.info("---NODE: SEARCH & INGEST---")
    question = state["question"]
    web_documents = searcher.search(question)
    if web_documents:
        ingestor.ingest_documents(web_documents)
    return {}

def generate_answer_node(state: AgentState) -> Dict[str, Any]:
    """Node 4: Generate the final, conversational answer."""
    logger.info("---NODE: GENERATE ADVISOR RESPONSE---")
    question = state["question"]
    user_profile = state["user_profile"]
    documents = state["documents"]
    generation = advisor.generate_response(question, user_profile, documents)
    
    history = state.get("conversation_history", [])
    history.append(f"User: {question}")
    history.append(f"Agent: {generation}")
    
    return {"generation": generation, "conversation_history": history}

# --- Conditional Edge Logic ---
def should_search_web(state: AgentState) -> str:
    """The conditional edge that directs the graph's flow."""
    logger.info("---ROUTING: Evaluating document relevance---")
    if state["is_relevant"]:
        logger.info(">>> Route: Documents are relevant. Proceeding to generate answer.")
        return "generate"
    else:
        logger.info(">>> Route: Documents are NOT relevant. Proceeding to web search.")
        return "search"

# --- Build and Compile the Graph ---
db_connection = sqlite3.connect("data/checkpoints.db", check_same_thread=False)
memory = SqliteSaver(db_connection)

builder = StateGraph(AgentState)

builder.add_node("reformulate_query", reformulate_query_node)
builder.add_node("transform_query", transform_query_node)
builder.add_node("route_documents", route_documents_node)
builder.add_node("search_and_ingest", search_and_ingest_node)
builder.add_node("generate_answer", generate_answer_node)

builder.set_entry_point("reformulate_query")
builder.add_edge("reformulate_query", "transform_query")
builder.add_edge("transform_query", "route_documents")
builder.add_conditional_edges(
    "route_documents",
    should_search_web,
    {"search": "search_and_ingest", "generate": "generate_answer"},
)
builder.add_edge("search_and_ingest", "generate_answer")
builder.add_edge("generate_answer", END)

app = builder.compile(checkpointer=memory)

# --- Interactive Test Harness (Updated to manage state correctly) ---
if __name__ == '__main__':
    print("--- InsuCompass AI Orchestrator ---")
    
    # --- Phase 1: Profile Building (This part remains the same) ---
    print("Phase 1: Building your health profile...")
    user_profile = {
    "zip_code": "30303",
    "county": "Fulton",
    "state": "Georgia",
    "age": 34,
    "gender": "Female", # Change to "Male" to test the other logic path
    "household_size": 1,
    "income": 65000,
    "employment_status": "employed_without_coverage",
    "citizenship": "US Citizen",
    "medical_history": None,
    "medications": None,
    "special_cases": None
    }
    profile_conversation_history = []
    for turn in range(10):
        question = profile_builder.get_next_question(user_profile)
        if question == "PROFILE_COMPLETE": break
        print(f"\nInsuCompass Agent: {question}")
        profile_conversation_history.append(f"Agent: {question}")
        user_answer = input("Your Answer > ")
        if user_answer.lower() == 'quit': exit()
        profile_conversation_history.append(f"User: {user_answer}")
        user_profile = profile_builder.update_profile_with_answer(
            user_profile, question, user_answer
        )
    print("\n--- Profile building complete! ---")
    print("\nFinal Profile:", json.dumps(user_profile, indent=2))
    
    # --- Phase 2: Q&A Session ---
    print("\n" + "#"*20 + " Q&A Session " + "#"*20)
    print("Your profile is complete. You can now ask me any questions.")
    
    thread_config = {"configurable": {"thread_id": "user-123-session-1"}}
    qna_history = []

    while True:
        user_question = input("\nYour Question > ")
        if user_question.lower() == 'quit': break
        
        inputs = {
            "question": user_question,
            "user_profile": user_profile,
            "conversation_history": qna_history # Pass the current history
        }
        
        print("\n--- InsuCompass is thinking... ---")
        final_state = {}
        for s in app.stream(inputs, config=thread_config):
            node_name = list(s.keys())[0]
            print(f"--- Executed Node: {node_name} ---")
            final_state.update(s)

        final_answer = final_state.get("generate_answer", {}).get("generation", "Sorry, I couldn't generate a response.")
        
        print("\n" + "="*20 + " FINAL ANSWER " + "="*20)
        print(final_answer)
        
        qna_history = final_state.get("generate_answer", {}).get("conversation_history", qna_history)

In [1]:
import logging
import json
import uuid
import sqlite3
from typing import List, Dict, Any
from typing_extensions import TypedDict

from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver

# Import all our custom agent and service classes
from insucompass.core.agents.profile_agent import profile_builder
from insucompass.core.agents.query_trasformer import QueryTransformationAgent
from insucompass.core.agents.router_agent import router
from insucompass.services.ingestion_service import IngestionService
from insucompass.core.agents.search_agent import searcher
from insucompass.core.agents.advisor_agent import advisor

from insucompass.services import llm_provider
from insucompass.prompts.prompt_loader import load_prompt
from insucompass.services.vector_store import vector_store_service

llm = llm_provider.get_gemini_llm()
retriever = vector_store_service.get_retriever()
transformer = QueryTransformationAgent(llm, retriever)
ingestor = IngestionService()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Unified LangGraph State Definition ---
class AgentState(TypedDict):
    user_profile: Dict[str, Any]
    user_message: str
    conversation_history: List[str]
    is_profile_complete: bool
    # Q&A specific fields
    standalone_question: str
    documents: List[Document]
    is_relevant: bool
    generation: str

# --- Graph Nodes ---

def profile_builder_node(state: AgentState) -> Dict[str, Any]:
    """A single turn of the profile building conversation."""
    logger.info("---NODE: PROFILE BUILDER---")
    profile = state["user_profile"]
    message = state["user_message"]
    history = state.get("conversation_history", [])

    if message == "START_PROFILE_BUILDING":
        agent_response = profile_builder.get_next_question(profile, [])
        new_history = [f"Agent: {agent_response}"]
        return {"conversation_history": new_history, "generation": agent_response, "user_profile": profile, "is_profile_complete": False}

    last_question = history[-1][len("Agent: "):] if history and history[-1].startswith("Agent:") else ""
    updated_profile = profile_builder.update_profile_with_answer(profile, last_question, message)
    agent_response = profile_builder.get_next_question(updated_profile, history + [f"User: {message}"])
    
    new_history = history + [f"User: {message}", f"Agent: {agent_response}"]
    
    if agent_response == "PROFILE_COMPLETE":
        logger.info("Profile building complete.")
        final_message = "Great! Your profile is complete. How can I help you with your health insurance questions?"
        new_history[-1] = f"Agent: {final_message}" # Replace "PROFILE_COMPLETE"
        return {"user_profile": updated_profile, "is_profile_complete": True, "conversation_history": new_history, "generation": final_message}
    
    return {"user_profile": updated_profile, "is_profile_complete": False, "conversation_history": new_history, "generation": agent_response}

def reformulate_query_node(state: AgentState) -> Dict[str, Any]:
    """Reformulates the user's question to be self-contained."""
    logger.info("---NODE: REFORMULATE QUERY---")
    question = state["user_message"]
    history = state["conversation_history"]
    user_profile = state["user_profile"]
    
    profile_summary = f"User profile context: State={user_profile.get('state')}, Age={user_profile.get('age')}, History={user_profile.get('medical_history')}"
    prompt = load_prompt("query_reformulator")
    history_str = "\n".join(history)
    
    full_prompt = f"{prompt}\n\n### User Profile Summary\n{profile_summary}\n\n### Conversation History:\n{history_str}\n\n### Follow-up Question:\n{question}"
    
    response = llm.invoke(full_prompt)
    standalone_question = response.content.strip()
    return {"standalone_question": standalone_question}

def retrieve_and_grade_node(state: AgentState) -> Dict[str, Any]:
    """Retrieves documents and grades them."""
    logger.info("---NODE: RETRIEVE & GRADE---")
    standalone_question = state["standalone_question"]
    documents = transformer.transform_and_retrieve(standalone_question)
    is_relevant = router.grade_documents(standalone_question, documents)
    return {"documents": documents, "is_relevant": is_relevant}

def search_and_ingest_node(state: AgentState) -> Dict[str, Any]:
    """Searches the web and ingests new info."""
    logger.info("---NODE: SEARCH & INGEST---")
    web_documents = searcher.search(state["standalone_question"])
    if web_documents:
        ingestor.ingest_documents(web_documents)
    return {}

def generate_answer_node(state: AgentState) -> Dict[str, Any]:
    """Generates the final answer."""
    logger.info("---NODE: GENERATE ADVISOR RESPONSE---")
    generation = advisor.generate_response(
        state["standalone_question"], state["user_profile"], state["documents"]
    )
    history = state["conversation_history"] + [f"User: {state['user_message']}", f"Agent: {generation}"]
    return {"generation": generation, "conversation_history": history}

# --- Conditional Edges ---
def should_search_web(state: AgentState) -> str:
    return "search" if not state["is_relevant"] else "generate"

# (CORRECTED) This is the function for the entry point conditional edge
def decide_entry_point(state: AgentState) -> str:
    """Decides the initial path based on profile completion status."""
    logger.info("---ROUTING: ENTRY POINT---")
    if state.get("is_profile_complete"):
        logger.info(">>> Route: Profile is complete. Starting Q&A.")
        return "qna"
    else:
        logger.info(">>> Route: Profile is not complete. Starting Profile Builder.")
        return "profile"

# --- Build the Graph ---
db_connection = sqlite3.connect("data/checkpoints.db", check_same_thread=False)
memory = SqliteSaver(db_connection)

builder = StateGraph(AgentState)

# (CORRECTED) Removed the faulty entry_router_node
builder.add_node("profile_builder", profile_builder_node)
builder.add_node("reformulate_query", reformulate_query_node)
builder.add_node("retrieve_and_grade", retrieve_and_grade_node)
builder.add_node("search_and_ingest", search_and_ingest_node)
builder.add_node("generate_answer", generate_answer_node)

# (CORRECTED) Set a conditional entry point
builder.set_conditional_entry_point(
    decide_entry_point,
    {
        "profile": "profile_builder",
        "qna": "reformulate_query"
    }
)

# Define graph edges
builder.add_edge("profile_builder", END) # A profile turn is one full loop. The state is saved, and the next call will re-evaluate at the entry point.
builder.add_edge("reformulate_query", "retrieve_and_grade")
builder.add_conditional_edges("retrieve_and_grade", should_search_web, {"search": "search_and_ingest", "generate": "generate_answer"})
builder.add_edge("search_and_ingest", "retrieve_and_grade") # Loop back to re-retrieve
builder.add_edge("generate_answer", END)

app = builder.compile(checkpointer=memory)

# --- Interactive Test Harness (CORRECTED) ---
if __name__ == '__main__':
    print("--- InsuCompass AI Unified Orchestrator Interactive Test ---")
    print("Type 'quit' at any time to exit.")

    test_thread_id = f"interactive-test-{uuid.uuid4()}"
    thread_config = {"configurable": {"thread_id": test_thread_id}}
    print(f"Using conversation thread_id: {test_thread_id}")

    # Initial state for a new user
    current_state = {
        "user_profile": {
            "zip_code": "90210", "county": "Los Angeles", "state": "California", "state_abbreviation": "CA",
            "age": 45, "gender": "Male", "household_size": 2, "income": 120000,
            "employment_status": "employed_with_employer_coverage", "citizenship": "US Citizen",
            "medical_history": None, "medications": None, "special_cases": None
        },
        "user_message": "START_PROFILE_BUILDING",
        "is_profile_complete": False,
        "conversation_history": [],
    }

    while True:
        print("\n" + "="*20 + " INVOKING GRAPH " + "="*20)
        print(f"Sending message: '{current_state['user_message']}'")
        
        # The graph is invoked with the current state
        final_state = app.invoke(current_state, config=thread_config)

        # Update our local state from the graph's final output
        current_state = final_state
        agent_response = current_state["generation"]
        
        print(f"\nInsuCompass Agent: {agent_response}")

        # Get the next input from the user
        if current_state["is_profile_complete"]:
            # If the last response was the completion message, prompt for a question
            if "profile is complete" in agent_response:
                 next_message = input("Your Question > ")
            else: # It was a Q&A response, so prompt for another question
                 next_message = input("Your Follow-up Question > ")
        else:
            next_message = input("Your Answer > ")

        if next_message.lower() == 'quit':
            print("Exiting test.")
            break
        
        # Prepare the state for the next turn
        current_state["user_message"] = next_message

2025-07-09 19:09:10,854 - INFO - Initialized LLM Provider: gemini-2.5-flash
2025-07-09 19:09:10,854 - INFO - Loading prompt 'profile_agent' from: /Users/nagurshareefshaik/Desktop/InsuCompass-AI/insucompass/prompts/profile_agent.txt
2025-07-09 19:09:10,855 - INFO - Loading prompt 'profile_updater' from: /Users/nagurshareefshaik/Desktop/InsuCompass-AI/insucompass/prompts/profile_updater.txt
2025-07-09 19:09:10,856 - INFO - ProfileBuilder initialized successfully with all prompts.
2025-07-09 19:09:10,867 - INFO - Loading prompt 'query_intent_classifier' from: /Users/nagurshareefshaik/Desktop/InsuCompass-AI/insucompass/prompts/query_intent_classifier.txt
2025-07-09 19:09:10,868 - INFO - Loading prompt 'query_transformer' from: /Users/nagurshareefshaik/Desktop/InsuCompass-AI/insucompass/prompts/query_transformer.txt
/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_41603/1771970172.py:15: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally

--- InsuCompass AI Unified Orchestrator Interactive Test ---
Type 'quit' at any time to exit.
Using conversation thread_id: interactive-test-89478936-0bc5-4b80-a549-840c5268e536

==================== INVOKING GRAPH ====================
Sending message: 'START_PROFILE_BUILDING'


2025-07-09 19:09:19,918 - INFO - LLM returned next step: '"Thank you for sharing those details! Now, let's gently move on to your health history. Could you please tell me about any significant medical conditions, diagnoses, or chronic illnesses you've experienced? There's no need to go into extreme detail, just the main points that might be relevant for your health plan."'



InsuCompass Agent: "Thank you for sharing those details! Now, let's gently move on to your health history. Could you please tell me about any significant medical conditions, diagnoses, or chronic illnesses you've experienced? There's no need to go into extreme detail, just the main points that might be relevant for your health plan."


2025-07-09 19:09:35,799 - INFO - ---ROUTING: ENTRY POINT---
2025-07-09 19:09:35,800 - INFO - >>> Route: Profile is not complete. Starting Profile Builder.
2025-07-09 19:09:35,803 - INFO - ---NODE: PROFILE BUILDER---


...


InsuCompass Agent: Thank you for confirming your medical history. Just one last area to cover: are there any major life events, planned medical procedures, or tobacco usage we should factor into your plan?


2025-07-09 19:09:45,074 - INFO - ---ROUTING: ENTRY POINT---
2025-07-09 19:09:45,075 - INFO - >>> Route: Profile is not complete. Starting Profile Builder.
2025-07-09 19:09:45,076 - INFO - ---NODE: PROFILE BUILDER---



==================== INVOKING GRAPH ====================
Sending message: 'Nope'


2025-07-09 19:09:46,483 - INFO - Successfully updated profile with user's answer.
2025-07-09 19:09:50,978 - INFO - LLM returned next step: 'PROFILE_COMPLETE'
2025-07-09 19:09:50,980 - INFO - Profile building complete.



InsuCompass Agent: Great! Your profile is complete. How can I help you with your health insurance questions?
Exiting test.
